In [76]:
import pandas as pd
import requests
import tensorflow as tf
from tensorflow import keras
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split
from random import sample
import json
# import utils
# from utils import *
# from utils import fetch_rawtext_from_wiki,preprocess_text,split_classify_text,split_train_test
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jannis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [96]:

def fetch_rawtext_from_wiki(subject='Maschinelles Lernen') :
     
    url = 'https://de.wikipedia.org/w/api.php'
    params = {
                'action': 'parse',
                'page': subject,
                'format': 'json',
                'prop':'text',
                'redirects':''
            }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    raw_html = data['parse']['text']['*']
    soup = BeautifulSoup(raw_html,'html.parser')
    soup.find_all('p')
    text = ''
    
    for p in soup.find_all('p'):
        text += p.text
    return text

def preprocess_text(text):
    newtext=re.sub("\[.*?\]","_",text)
    i=0
    newtext=newtext.replace("._","_.")
    while i<1:
        newtext=newtext.replace("._",".")
        i+=1
    return newtext

def classify_text(sen_text):
    is_claim=[]
    for i in sen_text:
        if "_" in i:
            is_claim.append(True)
        else:
            is_claim.append(False)
    df=pd.DataFrame()
    df["text"]=sen_text
    df["target"]=is_claim

    return df

def split_text(text):
    sen_text=text.split(".")
    return sen_text

def split_train_test(df):
    train_df, test_df = train_test_split(df,test_size=.10)

    return train_df,test_df

def remove_stopwords(text):
    stop = set(stopwords.words("german"))
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])
    
def fetch_from_fangcovid_local(number:int):
    text=" "
    save=os.getcwd()
    os.chdir("/Users/jannis/Desktop/TryVSCode/fang-covid-main/articles/")
    json_list=sample(range(1,40000),number)
    for i in json_list:
        try:
            f=open(str(i)+".json")
            df=json.load(f)
            f.close()
        except:
            raise FileExistsError
        text=text+df["article"]
    return text
def fetch_from_fangcovid_remote(number:int):
    json_list=sample(range(1,40000),number)
    df=pd.read_csv("https://github.com/justusmattern/fang-covid/blob/main/articles/0.json?raw=true")
    # for i in json_list:
    #     url="https://github.com/justusmattern/fang-covid/tree/main/articles"
    return df
# vectorize a text corpus by turning each text into a sequence of integers
# fit only to training

In [123]:

# Run Init Training/Test
articles=["Maschinelles Lernen", "Medizin","Wissenschaft","Krankheit","Prävention","Diagnose","Politik","COVID-19","COVID-19-Pandemie","Epidemie","Mykose","Sexuell übertragbare Erkrankung","Infektionskrankheit","Bundestag","Bundesrat","Zeitung","Rundfunk","Verlag","Politisches System der Bundesrepublik Deutschland","Politisches System","Massenmedien","Medienwissenschaft","Publikation"]
text="Auto"
for name in articles:
    raw=fetch_rawtext_from_wiki(name)
    text=text + raw

text_pre=preprocess_text(text)
text_split=split_text(text_pre)
df=classify_text(text_split)
df["text"]=df.text.map(remove_stopwords)
counter = counter_word(df.text)
num_unique_words=len(counter)
train_df,test_df = split_train_test(df)
print(num_unique_words)
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_df["text"]) 
train_sequences = tokenizer.texts_to_sequences(train_df['text'])
val_sequences = tokenizer.texts_to_sequences(test_df['text'])
train_padded = pad_sequences(train_sequences, maxlen=32, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=32, padding="post", truncating="post")
word_index = tokenizer.word_index
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])


19281


In [126]:
fang_text=fetch_from_fangcovid_local(5)
fang_pre=preprocess_text(fang_text)
df_fang=pd.DataFrame()
df_fang["sentence"]=split_text(fang_pre)
df_fang["tokenized"]=tokenizer.texts_to_sequences(df_fang["sentence"])
fang_padded = pad_sequences(df_fang["tokenized"], maxlen=32, padding="post", truncating="post")
df_fang["predictions"]=model.predict(fang_padded)

7/7 [==============================] - 1s 6ms/step


In [122]:
print(df_fang.sort_values("predictions"))
os.chdir("/Users/jannis/Desktop/TryVSCode/fang-covid-main")
df_fang.to_csv("results.csv")

                                              sentence  \
21    Der CSU-Politiker erklärte am Samstag:\nDer S...   
2    \n            Nach der Schließung etlicher eur...   
74         Erik Lesser (4/+2:12,3) landete auf Rang 16   
75    Philipp Horn (3/2:20,0), Roman Rees (2/2:34,9...   
140   Gemeint ist das Lockern des Lockdown für die ...   
..                                                 ...   
155  \nVor einigen Tagen hatte die französische Reg...   
175   Zur Finanzierung sind in diesem Jahr 218 Mill...   
135   542 waren es auch im jüngsten Bulletin des Zi...   
37                                 ), Robin Becker (45   
105  \nBob: Laura Nolte (Winterberg) hat sich gleic...   

                                             tokenized  predictions  
21   [917, 253, 80, 2975, 92, 4448, 383, 23, 10019,...     0.001278  
2    [1741, 1351, 1446, 758, 279, 444, 3701, 8456, ...     0.001279  
74                   [115, 6, 165, 63, 189, 8035, 166]     0.001281  
75   [63, 6, 108, 101, 

In [121]:
os.getcwd()

'/Users/jannis/Desktop/TryVSCode/fang-covid-main/articles'

In [114]:
# df_fang=pd.DataFrame()
# df_fang["text"]=decode(fang_padded)
# df_fang["prediction"]=fang_predictions
# print(df_fang)


TypeError: unhashable type: 'numpy.ndarray'

In [124]:
# Create LSTM model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have 
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=32))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.


model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 32, 32)            616992    
                                                                 
 lstm_2 (LSTM)               (None, 64)                24832     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 641,889
Trainable params: 641,889
Non-trainable params: 0
_________________________________________________________________


In [125]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)
model.fit(train_padded, train_df["target"], epochs=20, validation_data=(val_padded, test_df["target"]), verbose=2)

Epoch 1/20


/Users/jannis/opt/anaconda3/envs/masterth-env/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


141/141 - 6s - loss: 0.5105 - accuracy: 0.7859 - val_loss: 0.4831 - val_accuracy: 0.7996 - 6s/epoch - 44ms/step
Epoch 2/20
141/141 - 3s - loss: 0.3450 - accuracy: 0.8515 - val_loss: 0.5179 - val_accuracy: 0.7735 - 3s/epoch - 23ms/step
Epoch 3/20
141/141 - 3s - loss: 0.1277 - accuracy: 0.9565 - val_loss: 0.6583 - val_accuracy: 0.7836 - 3s/epoch - 23ms/step
Epoch 4/20
141/141 - 3s - loss: 0.0540 - accuracy: 0.9855 - val_loss: 0.8680 - val_accuracy: 0.7816 - 3s/epoch - 21ms/step
Epoch 5/20
141/141 - 3s - loss: 0.0365 - accuracy: 0.9911 - val_loss: 0.9193 - val_accuracy: 0.7735 - 3s/epoch - 21ms/step
Epoch 6/20
141/141 - 3s - loss: 0.0300 - accuracy: 0.9917 - val_loss: 1.0820 - val_accuracy: 0.7595 - 3s/epoch - 21ms/step
Epoch 7/20
141/141 - 3s - loss: 0.0290 - accuracy: 0.9931 - val_loss: 1.0956 - val_accuracy: 0.7655 - 3s/epoch - 22ms/step
Epoch 8/20
141/141 - 3s - loss: 0.0217 - accuracy: 0.9940 - val_loss: 1.2691 - val_accuracy: 0.7515 - 3s/epoch - 21ms/step
Epoch 9/20
141/141 - 3s - l

In [89]:
text=fetch_from_fangcovid(5)
print(len(text))

17574


In [65]:
df=pd.read_csv("https://raw.githubusercontent.com/justusmattern/fang-covid/main/articles/20000.json")


ParserError: Error tokenizing data. C error: Expected 2 fields in line 7, saw 46


Papst Franziskus unterstützt weiterhin die globale Corona-Panikmache. Täglich prasseln sich an Dramatik überbietende „Schreckensmeldungen“ auf die Bevölkerung nieder, die bei näherem Hinsehen meist die Verhältnismäßigkeit vermissen lassen.
Papst Franziskus ließ nun verkünden, daß es auch zu Weihnachten keine Papstmessen geben werde. Obwohl die Corona-Medien die Nachricht bereitwillig in ihr tägliches Corona-Panikpensum aufnehmen, ist sie keine Sensation.
Eine Sensation wäre es gewesen, wenn der Papst angekündigt hätte, endlich wieder die Zelebration öffentlicher Gottesdienste aufzunehmen. Unterdessen erlebt die Kirche im Verhältnis die größte Fahnenflucht ihrer Geschichte.
Das vatikanische Staatssekretariat teilte gestern in einer Note allen beim Heiligen Stuhl akkreditierten Botschaftern mit, daß es zu Weihnachten, dem Geburtsfest Jesu Christi, keine öffentlichen päpstlichen Meßfeiern und protokollarischen Zeremonien geben wird.
Auf den ersten Blick klingt die die Ankündigung des vati

In [59]:
df=fetch_from_fangcovid(0)

ParserError: Error tokenizing data. C error: Expected 2 fields in line 6, saw 3


In [ ]:
def tokenize_text(text):
    nlp = spacy.load("de_core_news_md")
    doc = nlp(sentences[0])
    print(doc.text)
    for token in doc:
        print(token.text, token.pos_, token.dep_)